Copyright (C) 2023 Sean Ming-Fong Sie <seansie07@gmail.com>

This work is licensed under the Creative Commons Attribution-NonCommercial
4.0 International License. To view a copy of this license, visit
http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

In [1]:
from pyqubo import Spin
# set S = {4, 2, 7, 1}
s1, s2, s3, s4 = Spin("s1"), Spin("s2"), Spin("s3"), Spin("s4")
# Quadratic Unconstrained Binary Optimization（QUBO）for **2 as Hamiltonian 
H = (4*s1 + 2*s2 + 7*s3 + s4)**2
print("QUBO Hamiltonian:",H)
# Compile Quantum Computer or Classical Binary Optimization Solver as Hamiltonian to a model
model = H.compile()
print("Model:", model)
# Convert QUBO to Ising Hamiltonian, and get the offset as constant
qubo, offset = model.to_qubo()
print("QUBO Result:",qubo)
print("Offset:", offset)

QUBO Hamiltonian: ((Spin('s4') + (7.000000 * Spin('s3')) + (4.000000 * Spin('s1')) + (2.000000 * Spin('s2'))) * (Spin('s4') + (7.000000 * Spin('s3')) + (4.000000 * Spin('s1')) + (2.000000 * Spin('s2'))))
Model: <cpp_pyqubo.Model object at 0x7fd6b0807370>
QUBO Result: {('s1', 's1'): -160.0, ('s4', 's2'): 16.0, ('s3', 's1'): 224.0, ('s2', 's2'): -96.0, ('s4', 's1'): 32.0, ('s1', 's2'): 64.0, ('s3', 's2'): 112.0, ('s3', 's3'): -196.0, ('s4', 's4'): -52.0, ('s4', 's3'): 56.0}
Offset: 196.0


In [2]:
import neal
sampler = neal.SimulatedAnnealingSampler() # Simulated Annealing Sampler using D-Wave
print("Sampler:", sampler)
bqm = model.to_bqm() # Convert QUBO to Binary Quadratic Model
print("Binary Quadratic Model:", bqm)
sampleset = sampler.sample(bqm, num_reads=10) # Retrieve 10 samples
print("Sample Set:", sampleset)
decoded_samples = model.decode_sampleset(sampleset) # Decode the sampleset
print("Decoded Sample:", decoded_samples)
best_sample = min(decoded_samples, key=lambda x: x.energy) # Retrieve the best sample
print("Best Sample:", best_sample)
best_sample.sample # doctest: +SKIP, Retrieve the best sample's value
print("Best Sample:", best_sample.sample)

Sampler: <dwave.samplers.sa.sampler.SimulatedAnnealingSampler object at 0x7fd690ebdbb0>
Binary Quadratic Model: BinaryQuadraticModel({'s4': -52.0, 's2': -96.0, 's3': -196.0, 's1': -160.0}, {('s2', 's4'): 16.0, ('s3', 's4'): 56.0, ('s3', 's2'): 112.0, ('s1', 's4'): 32.0, ('s1', 's2'): 64.0, ('s1', 's3'): 224.0}, 196.0, 'BINARY')
Sample Set:   s1 s2 s3 s4 energy num_oc.
0  1  1  0  1    0.0       1
1  1  1  0  1    0.0       1
3  1  1  0  1    0.0       1
4  1  1  0  1    0.0       1
5  1  1  0  1    0.0       1
6  0  0  1  0    0.0       1
7  0  0  1  0    0.0       1
8  0  0  1  0    0.0       1
9  1  1  0  1    0.0       1
2  1  1  0  0    4.0       1
['BINARY', 10 rows, 10 samples, 4 variables]
Decoded Sample: [DecodedSolution({s3:1, s4:0, s2:0, s1:0}, energy=0.000000), DecodedSolution({s1:0, s2:0, s4:0, s3:1}, energy=0.000000), DecodedSolution({s3:1, s4:0, s2:0, s1:0}, energy=0.000000), DecodedSolution({s3:0, s4:1, s2:1, s1:1}, energy=0.000000), DecodedSolution({s1:1, s2:1, s4:1, s3

In [5]:
import pandas as pd
from pyqubo import Binary

# 指定CSV檔案的路徑
file_path = "../data_p/data.address.csv"

# 使用pandas的read_csv函數讀取CSV檔案
df = pd.read_csv(file_path)
print(df)

# 選擇要用於構建QUBO的列名，並且加上 class
selected_columns = df.columns[1:10].tolist() + ["class"]

# 從DataFrame中選擇所需的列
selected_data = df[selected_columns].copy()

# 預處理步驟
# 例如，使用中位數進行二元化
for col in selected_columns:
    if selected_data[col].dtype != 'object':
        median_val = selected_data[col].median()
        selected_data[col] = selected_data[col].apply(lambda x: 1 if x > median_val else 0)

# 創建QUBO模型
qubo_model = 0
for col in selected_columns:
    qubo_model += sum(selected_data[col]) * Binary(col)

# 添加特徵間的交互項（可選）
for i in range(len(selected_columns)):
    for j in range(i+1, len(selected_columns)):
        col_i = selected_columns[i]
        col_j = selected_columns[j]
        qubo_model += Binary(col_i) * Binary(col_j) * sum(selected_data[col_i] * selected_data[col_j])

# 添加偏移量以使所有特徵都為0時，Hamiltonian的值為100
desired_offset = 100
qubo_model += desired_offset

# 編譯模型
compiled_model = qubo_model.compile()

# 轉換為QUBO並獲取偏移量
qubo, offset = compiled_model.to_qubo()

# 顯示結果
print("QUBO Hamiltonian:", qubo_model)
print("Model:", compiled_model)
print("QUBO Result:", qubo)
print("Offset:", offset)


    n_tx  total_days  total_spent_btc  total_received_btc  total_spent_usd  \
0    1.0         1.0         0.012863            0.012863         9.529224   
1    1.0         1.0         0.000275            0.000275         0.203728   
2    1.0         1.0         0.163000            0.163000       112.258102   
3    3.0         1.0         0.093739            0.093739        64.558030   
4    3.0         1.0         0.000300            0.000300         0.206610   
..   ...         ...              ...                 ...              ...   
87   1.0         1.0         1.130100            1.130100       778.299884   
88   1.0         1.0         0.084900            0.084900        62.896380   
89   3.0         2.0         0.004823            0.004823         3.428225   
90   1.0         1.0         0.114600            0.114600        84.899001   
91   1.0         1.0         1.937600            1.937600      1362.588108   

    total_received_usd  mean_balance_btc  std_balance_btc  mean

In [4]:
import neal
from pyqubo import Binary

# 編譯為模型
compiled_model = qubo_model.compile()
print("Model:", compiled_model)

# 將 QUBO 轉換為二元二次模型
bqm = compiled_model.to_bqm()
print("Binary Quadratic Model:", bqm)

# 使用模擬退火算法
sampler = neal.SimulatedAnnealingSampler()
print("Sampler:", sampler)

# 採樣10次
num_reads = 10
sampleset = sampler.sample(bqm, num_reads=num_reads)
print("Sample Set:", sampleset)

# 解碼採樣結果
decoded_samples = compiled_model.decode_sampleset(sampleset)
print("Decoded Sample:", decoded_samples)

# 尋找能量最低的樣本
best_sample = min(decoded_samples, key=lambda x: x.energy)
print("Best Sample:", best_sample)

# 獲取最佳樣本的值
best_sample_value = best_sample.sample
print("Best Sample:", best_sample_value)


Model: <cpp_pyqubo.Model object at 0x7fd6b1031230>
Binary Quadratic Model: BinaryQuadraticModel({'std_balance_usd': 13.0, 'total_received_usd': 46.0, 'std_balance_btc': 14.0, 'total_received_btc': 46.0, 'total_spent_usd': 46.0, 'total_spent_btc': 46.0, 'mean_balance_usd': 46.0, 'total_days': 4.0, 'class': 16.0, 'mean_balance_btc': 46.0}, {('total_received_usd', 'std_balance_usd'): 8.0, ('std_balance_btc', 'std_balance_usd'): 13.0, ('std_balance_btc', 'total_received_usd'): 8.0, ('total_received_btc', 'std_balance_usd'): 8.0, ('total_received_btc', 'total_received_usd'): 46.0, ('total_received_btc', 'std_balance_btc'): 8.0, ('total_spent_usd', 'std_balance_usd'): 8.0, ('total_spent_usd', 'total_received_usd'): 46.0, ('total_spent_usd', 'std_balance_btc'): 8.0, ('total_spent_usd', 'total_received_btc'): 46.0, ('total_spent_btc', 'std_balance_usd'): 8.0, ('total_spent_btc', 'total_received_usd'): 46.0, ('total_spent_btc', 'std_balance_btc'): 8.0, ('total_spent_btc', 'total_received_btc'):